# Aprendizaje basado en instancias

En esta sesión de trabajo con Jupyter aplicaremos los conceptos presentados en el módulo sobre aprendizaje basado en instancias al conjunto de datos Iris.

## Procesado de los datos

El conjunto de [datos Iris](https://es.wikipedia.org/wiki/Iris_flor_conjunto_de_datos) contiene datos sobre cuatro características morfológicas de tres especies de la flor Iris: Iris setosa, Iris virginica e Iris versicolor. En concreto hay 50 muestras de cada especie. Para cada una de ellas se tiene información sobre el largo y el ancho del pétalo y el sépalo, medidos en centímetros.

Lo primero que vamos a hacer es cargar el conjunto de datos Iris incluido en *Scikit-learn*, usando la función [`load_iris()`](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html) incluido en la librería `sklearn.datasets`

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()

Este conjunto de datos es un diccionario con varios campos:
* `data`: Es el conjunto de datos, se trata de un array en el que cada componente es un array con las características de cada instancia.
* `target`: Es el conjunto de valores de clasificación para cada instancia. Es un array del mismo tamaño que `data`, en el que se indica el valor de clasificación de cada instancia, en el mismo orden en que éstas se encuentran en el array `data`.
* `target_names`: Es un array con los nombres de cada valor de clasificación.
* `feature_names`: Es un array con los nombres de cada característica.
* `DESCR`: Es una descripción del conjunto de datos.


El valor `i`-ésimo del array de datos y su valor de clasificación se pueden obtener con la siguiente función:

In [ ]:
def datoIesimo(i):
    print("Dato (",i,"): ",iris['data'][i])
    print("  Clasificación: ",iris['target'][i])

Los valores de clasificación almacenados en el array `target` son valores numéricos contados desde 0 en adelante y se corresponden con la clasificación almacenada en el array `target_names`. Es decir el valor de clasificación `j` se corresponde con la clasificación `target_names[j]`. Teniendo esto en cuenta podemos mejorar la función anterior para proporcionar esta información:

In [ ]:
def datoIesimo(i):
    print("Dato (",i,"): ",iris['data'][i])
    print("  Clasificación: ",iris['target_names'][iris['target'][i]])

De la misma forma, los valores que contiene cada dato se corresponden con los valores de las características, en el mismo orden en que aparecen en el array `feature_names`. Finalmente, la función anterior se puede mejorar asociando cada valor del dato con la característica correspondiente:

In [ ]:
def datoIesimo(i):
    print("Características del dato número ",i,":")
    for j in range(4):
        print(" ",iris['feature_names'][j],": ",iris['data'][i][j])
    print("  Clasificación: ",iris['target_names'][iris['target'][i]])

El último valor devuelto por la función `load_iris()` es una descripción del conjunto de datos.

In [ ]:
print(iris['DESCR'])

Almacenamos cada uno de los campos que resultan de interés para este ejercicio en variables distintas.

In [ ]:
X_data, y_data, X_names, y_names = \
    iris.data, iris.target, iris.feature_names, iris.target_names

Podemos visualizar el conjunto de datos usando la librería `matplotlib`. Para esto, consideramos un par de características 
(por ejemplo las dos primeras) y presentamos en un gráfico 2D un conjunto de puntos que representan cada uno de los ejemplos del conjunto de datos. Para diferenciar los valores de clasificación usamos distintas formas y colores para cada uno de ellos: cuadrados rojos para el primer valor de clasificación (Iris setosa), círculos verdes para el segundo valor de clasificación (Iris versicolor) y rombos azules para el tercer valor de clasificación (Iris virgínica).

Cargamos la librería [`pyplot`](https://matplotlib.org/api/pyplot_api.html) de la biblioteca `matplotlib` con el nombre `plt`:

In [ ]:
from matplotlib import pyplot as plt

El proceso para dibujar los datos consiste en limpiar el lienzo (`plt.clf()`) y para cada terna formada por el valor de clasificación (`range(len(y_names))`), la forma (`"soD"`) y el color (`"rgb"`) deseados, recoger en las variables `xs` e `ys` los valores de las características a representar de todos los datos con el valor de clasificación considerado. Estos puntos se representan con el método (`plt.scatter`) con la forma y el color considerados. Finalmente etiquetamos los ejes horizontal y vertical con los nombres de las características representadas.

El siguiente trozo de código sirve para visualizar como se distribuyen los datos con respecto a los valores de las características longitud de sépalo y anchura de sépalo:

In [ ]:
plt.clf()
for tipo,marca,color in zip(range(len(y_names)),"soD","rgb"):
    xs=X_data[y_data==tipo,0]
    ys=X_data[y_data==tipo,1]
    plt.scatter(xs,ys,marker=marca,c=color)
plt.xlabel("Longitud de sépalo")
plt.ylabel("Anchura de sépalo")
plt.show()

En este gráfico se puede observar que los datos de las flores de la variedad setosa se pueden separar fácilmente de los otros, que se mezclan un poco. 

Este proceso se puede definir como el efecto de una función:

In [ ]:
def representacion_grafica(datos,caracteristicas,objetivo,clases,c1,c2):
    for tipo,marca,color in zip(range(len(clases)),"soD","rgb"):
        xs = datos[objetivo == tipo,c1]
        ys = datos[objetivo == tipo,c2]
        plt.scatter(xs,ys,marker=marca,c=color)
    plt.xlabel(caracteristicas[c1])
    plt.ylabel(caracteristicas[c2])

Podemos utilizar esta función para representar gráficamente la distribución de los datos con respecto a cada pareja de características. Eso se consigue dividiendo el lienzo en seis trozos, en 2 filas y 3 columnas con las correspondientes llamadas al método `plt.subplot`:

In [ ]:
def representacion_conjunta():
    plt.clf()
    plt.rcParams["figure.figsize"] = [18,12]
    plt.subplot(231)
    representacion_grafica(X_data,X_names,y_data,y_names,0,1)
    plt.subplot(232)
    representacion_grafica(X_data,X_names,y_data,y_names,0,2)
    plt.subplot(233)
    representacion_grafica(X_data,X_names,y_data,y_names,0,3)
    plt.subplot(234)
    representacion_grafica(X_data,X_names,y_data,y_names,1,2)
    plt.subplot(235)
    representacion_grafica(X_data,X_names,y_data,y_names,1,3)
    plt.subplot(236)
    representacion_grafica(X_data,X_names,y_data,y_names,2,3)
    plt.subplots_adjust(wspace=0.2,hspace=0.2)
    plt.show()

representacion_conjunta()

De los gráficos anteriores se puede deducir que las características que mejor separan los datos (y por tanto las que mejor sirven para clasificarlos) son la longitud y anchura del pétalo. 

## Aprendizaje basado en instancias: Algoritmo k-NN

Vamos a aplicar el algoritmo k-NN al conjunto de datos Iris para clasificar nuevas instancias. Para que esto tenga sentido, tendremos que dividir el conjunto de datos original en dos trozos: un conjunto de entrenamiento para el aprendizaje y un conjunto de prueba para la evaluación. Esto lo conseguimos con la función `train_test_split` de la librería `sklearn.model_selection`.

In [ ]:
from sklearn.model_selection import train_test_split

Para usar esta función debemos proporcionar el array de datos original `X_data`, el array de valores de clasificación `y_data`, el tamaño de uno de los conjuntos resultado, `test_size` o `train_size`, que puede ser un valor entero indicado un valor absoluto del número de ejemplos o un número real entre 0 y 1, que representa un porcentaje del conjnto de datos original. Esta función realiza una selección aleatoria de índices en el rango de los arrays `X_data` e `y_data`, respetando la proporción indicada. Una vez hecho esto, devuelve los arrays de datos resultado de seleccionar en `X_data` e `y_data` los valores asociados a dichos índices, y los restantes. El parámetro `stratify` sirve para indicar con respecto a qué valores se quieren estratificar los datos (normalmente el valor de clasificación).

In [ ]:
X_train, X_test, y_train, y_test = \
  train_test_split(X_data,y_data,test_size = 0.33,
                   random_state=4861,stratify=y_data)

Podemos ver gráficamente cómo se han distribuido los datos en los conjuntos de entrenamiento y prueba, representando simultáneamente estos conjuntos de datos junto con el conjunto de datos original.

In [ ]:
plt.rcParams["figure.figsize"] = [18,6]
plt.clf()
splt = plt.subplot(131)
splt.set_xlim(4,8)
splt.set_ylim(1.8,4.5)
splt.set_title("Conjunto de datos")
representacion_grafica(X_data,X_names,y_data,y_names,0,1)
splt = plt.subplot(132)
splt.set_xlim(4,8)
splt.set_ylim(1.8,4.5)
splt.set_title("Conjunto de entrenamiento")
representacion_grafica(X_train,X_names,y_train,y_names,0,1)
splt = plt.subplot(133)
splt.set_xlim(4,8)
splt.set_ylim(1.8,4.5)
splt.set_title("Conjunto de prueba")
representacion_grafica(X_test,X_names,y_test,y_names,0,1)
plt.show()

El algoritmo de clasificación **k**-*NN* se encuentra en la librería `sklearn.neighbors` implementado mediante la clase `KNeighborsClassifier`.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

Creamos una instancia de este algoritmo con un valor de **k** igual a 5, este valor se indica con el parámetro `n_neighbors`. Para indicar la distancia a utilizar usamos el paramétro `p`, que será el valor del que dependerá la distancia Minkowski:
$$d_p(\textbf{x},\textbf{y})=(\sum_{i=1}^n(x_i-y_i)^p)^{1/p}$$

Como recordaréris, un valor de `p` igual a `1` se corresponde con el uso de la distancia Manhattan y un valor de `p` igual a `2` se corresponde con el uso de la distancia euclídea. También se puede usar otra distancia, prorcionándola como valor del parámetro `metric`.

In [ ]:
knn1 = KNeighborsClassifier(n_neighbors=5,p=2)

Todos las implementaciones de algoritmos de aprendizaje en *Scikit-learn* admiten una serie de métodos con la misma funcionalidad:
* `fit`: Sirve para entrenar el modelo asociado, es decir aprender los parámetros que sirven para ajustarlo al conjunto de datos. Recibe como argumentos el array de características de los ejemplos y el de sus valores de clasificación.
* `predict`: Sirve para evaluar el modelo entrenado sobre un array de ejemplos. Recibe como argumento el array de características de los ejemplos y devuelve el array con sus valores de clasificación según el modelo.
* `score`: Mide el rendimiento del modelo entrenado sobre un array de ejemplos. Recibe como argumentos el array de características de los ejemplos y el de sus valores de clasificación. Devuelve el rendimiento del modelo sobre dicho conjunto de ejemplos.

Una vez creada la instancia del algoritmo de aprendizaje, hay que entrenarla con el conjunto de ejemplos de entrenamiento. En este caso, el entrenamiento sirve para organizar los datos de la mejor forma posible para calcular las distancias en las que se basa el proceso de clasificación.

In [ ]:
knn1.fit(X_train,y_train)

Al entrenar el modelo, podemos ver los valores de los parámetros de los que depende el proceso de entrenamiento. Algunos de ellos (`algorithm`, `leaf_size` y `n_jobs`) son relativos al proceso de cálculo de las distancias necesarias para determinar los mejores vecinos, es decir, parámetros del algoritmo de entrenamiento; y otros (`metric`, `metric_params`, `n_neighbors`, `p` y `weights`) son relativos al proceso de selección de los mejores vecinos y de clasificación, es decir, parámetros del modelo de clasificación.

Veamos ahora el valor de clasificación de los ejemplos del conjunto de prueba.

In [ ]:
knn1.predict(X_test)

Si comparamos este array con el de valores de clasificación del conjunto de prueba, `y_test`, podemos calcular el rendimiento del modelo.

In [ ]:
y_test

In [ ]:
y_clas = knn1.predict(X_test)
y_clas[y_clas != y_test]

Esto es precisamente lo que calcula el tercero de los métodos que hemos mencionado antes, `score`.

In [ ]:
knn1.score(X_test,y_test)

Antes se ha visto otro parámetro, `weights`, del modelo de clasificación. Se trata de un forma de ponderar los valores de clasificación de los mejores vecinos. El valor por defecto es `uniform`, que indica que los valores de clasificación de los mejores vecinos se tratan de forma uniforme, sin dar más importancia a unos que a otros. El valor `distance` indica que se debe dar más importancia a los valores de clasificación de los mejores vecinos más cercanos. Esto puede afectar al resultado de la clasificación, haciendo variar el rendimiento.

Para usar este parámetro, tenemos que indicar su valor en el momento de crear la instancia del algoritmo de clasificación:

In [ ]:
knn2 = KNeighborsClassifier(n_neighbors=5,p=2,weights='distance')

Entrenamos la nueva instancia con el mismo conjunto de ejemplos:

In [ ]:
knn2.fit(X_train,y_train)

Y vamos a ver si hay algún cambio en el rendimiento del modelo obtenido

In [ ]:
knn2.score(X_test,y_test)

Veamos ahora el efecto que produce cambiar la distancia. Vamos a construir una instancia del algoritmo de clasificación usando la distancia Manhattan. Para esto, solo hay que indicar el valor `1` para el parámetro `p`.

In [ ]:
knn3 = KNeighborsClassifier(n_neighbors=5,p=1)

Entrenamos la instancia creada con el conjunto de entrenamiento

In [ ]:
knn3.fit(X_train,y_train)

Y analizamos el rendimiento del modelo obtenido

In [ ]:
knn3.score(X_test,y_test)

En este caso el resultado es peor que con la distancia euclídea. En este caso, dado que las características miden propiedades similares (longitudes) era de esperar que la distancia euclídea funcionase mejor que la Manhattan. Esta última resulta más útil cuando las características miden propiedades que no tienen nada en común.

## Normalización

El último factor que vamos a ver de los influyen en el modelo **k**-*NN* es la normalización. En este caso, las medidas de longitud de pétalos y sépalos tienen mayor rango que las medidas de anchura. Eso podría influir en las distancias entre los ejemplos. Para evitarlo, podemos normalizar los datos. *Scikit-learn* proporciona varias formas de normalizar un conjunto de datos. Aquí usamos la implementada en la clase `StandardScaler` de la librería `sklearn.preprocessing`:

In [ ]:
from sklearn.preprocessing import StandardScaler

Al igual que los modelos de clasificación, el normalizador se 'entrena' con el conjunto de datos de entrenamiento, para determinar su media y su varianza. Esto se consigue con el método `fit`.

In [ ]:
normalizador = StandardScaler().fit(X_train)

A partir de los valores aprendidos, se pueden modificar los datos originales de forma que el resultado quede con un valor de la media aproximadamente igual a 0 y un valor de la varianza aproximadamente igual a 1. Esto se consigue con el médodo `transform`:

In [ ]:
Xn_train = normalizador.transform(X_train)
Xn_test = normalizador.transform(X_test)

Es muy importante que los ejemplos del conjunto de prueba se normalicen de la misma forma que los del conjunto de entrenamiento, para que el modelo aprendido funcione correctamente.

Los valores aprendidos por el normalizador se almacenan en los parámetros `mean_` para las medias de las características, `var_` para las varianzas y `scale_` para los factores multiplicativos que cambian la escala.

In [ ]:
normalizador.mean_

In [ ]:
normalizador.var_

In [ ]:
normalizador.scale_

In [ ]:
from scipy import mean,var
print("Media de los datos originales: ",mean(X_train[:,0]))
print("Media de los datos normalizados: ",mean(Xn_train[:,0]))
print("Varianza de los datos originales: ",var(X_train[:,0]))
print("Varianza de los datos normalizados: ",var(Xn_train[:,0]))

Podemos representar gráficamente los datos antes y después de la normalización para ver las diferencias

In [ ]:
plt.rcParams["figure.figsize"] = [18,6]
plt.clf()
splt = plt.subplot(121)
splt.set_xlim(4,8)
splt.set_ylim(1.8,4.5)
splt.set_title("Conjunto de datos sin normalizar")
representacion_grafica(X_data,X_names,y_data,y_names,0,1)
splt = plt.subplot(122)
splt.set_xlim(-3.5,3.5)
splt.set_ylim(-3.5,3.5)
splt.set_title("Conjunto de datos normalizados")
Xn_data = normalizador.transform(X_data)
representacion_grafica(Xn_data,X_names,y_data,y_names,0,1)

Con estos nuevos datos, entrenamos una nueva instancia del algoritmo **k**-*NN*:

In [ ]:
knn4 = KNeighborsClassifier(n_neighbors=5,p=2,weights='distance')
knn4.fit(Xn_train,y_train)
knn4.score(Xn_test,y_test)